# CSI5155 Machine Learning Assignment 2

This notebook serves as a starting point to firstly evaluate the predictions from models which we trained in assignment 1, and precisely to calculate the SHAP values for each selected classifiers.

# Installing Packages and Prerequisites

In [1]:
%pip install numpy
%pip install joblib
%pip install shap
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np;
import constants;
import os;
from fileOrganizer import unpack;
import random;
from explainer import explainer;
import gc;

c:\Users\kelvi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model selection and Applying SHAP

According to assignment 1 (as shown in this notebook `CSI5155 Assignment 1 Evaluation Part - Kelvin Mock 300453668.ipynb` and the report `CSI5155 Assignment 1 Report.pdf`), we compared the Areas Under the Curves (AUCs) among 6 models. We also concluded in the report that the AUC is a great metric to inference the overall accuracy of a model. For simplicity, we only consider the **original** models where sampling techniques are not applied. We have the following conclusions:
- Based on the Chocolate dataset, the best classifier (with the largest AUC) is the Decision Tree classifier; and, 
- Based on the Magic Mushroom dataset, the best classifier (with the largest AUC) is the Multi-Layer Perceptron (MLP) classifier,

whereas,

- Based on the Chocolate dataset, the worst classifier (with the lowest AUC) is the Support Vector Machine (SVM) classifier; and, 
- Based on the Magic Mushroom dataset, the worst classifier (with the lowest AUC) is also the SVM classifier. 

## Loading the Models from files

In [3]:
def load_models(path: str):
    # Choco Best: Decision Tree
    choco_bestModel = unpack(os.path.join(path, constants.filepaths["choc_posttrained_decisionTree"]));
    # Choco Worst: SVM
    choco_worstModel = unpack(os.path.join(path, constants.filepaths["choc_posttrained_SVC"]));
    # Mushroom Best: MLP
    mush_bestModel = unpack(os.path.join(path, constants.filepaths["mushrooms_posttrained_MLP"]));
    # Mushroom Worst: SVM
    mush_worstModel = unpack(os.path.join(path, constants.filepaths["mushrooms_posttrained_SVC"]));
    return choco_bestModel, choco_worstModel, mush_bestModel, mush_worstModel;

try:
    choco_bestModel, choco_worstModel, mush_bestModel, mush_worstModel = load_models(constants.ASM1_DIR);
except FileNotFoundError:
    # alt path for remote / github action
    choco_bestModel, choco_worstModel, mush_bestModel, mush_worstModel = load_models(constants.ASM1_DIR_ALT);

### Inspecting the Imported Models

In [4]:
choco_bestModel

RandomizedSearchCV(estimator=DecisionTreeClassifier(criterion='entropy',
                                                    splitter='random'),
                   param_distributions={'max_depth': array([11, 10, 18,  9,  1,  2, 15, 17, 11,  1,  5,  5, 14, 10,  1,  6, 15,
       12, 19, 12,  8, 13, 15,  6, 16, 14, 11,  7,  8, 15,  5, 10,  5, 10,
       19, 16, 16,  1,  5, 17, 17,  3,  9, 10,  9,  6, 14, 11,  9, 15,  8,
       14, 12, 19,  2,  4,  3, 10, 13,  7,  4,  8, 14, 12,  3, 19, 12, 10,
        4, 11, 16, 19, 16,  6, 16, 19, 14,  5,  8, 16, 14, 14, 14, 15,  3,
       12, 15,  5, 12,  7, 10, 17,  8, 18, 10, 13,  9, 13,  5, 16],
      dtype=int64),
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001CB1ED67BE0>},
                   random_state=42)

Since the Decision Tree classifier has been optimized by RandomizedSearchCV, we need to extract the estimator. 

In [5]:
choco_bestModel = choco_bestModel.best_estimator_;
choco_bestModel

DecisionTreeClassifier(criterion='entropy', max_depth=14,
                       min_samples_split=0.9556428757689246, splitter='random')

In [6]:
choco_worstModel

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001CB56B1ACB0>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CB56B45090>,
                                        'gamma': ['scale', 'auto',
                                                  0.028189765546689366,
                                                  0.09443747650409191,
                                                  0.07873295925515456,
                                                  0.06200241739172151,
                                                  0.029597687946635268],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=42)

Since the SVM classifier has been optimized by RandomizedSearchCV, we need to extract the estimator. 

In [7]:
choco_worstModel = choco_worstModel.best_estimator_;
choco_worstModel

SVC(C=3.4370861113902182, degree=4, gamma=0.06200241739172151, kernel='linear')

In [8]:
mush_bestModel

MLPClassifier(hidden_layer_sizes=3769, learning_rate='adaptive', max_iter=500)

In [9]:
mush_worstModel

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001CB56B47FA0>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CB56B476D0>,
                                        'gamma': ['scale', 'auto',
                                                  0.028189765546689366,
                                                  0.09443747650409191,
                                                  0.07873295925515456,
                                                  0.06200241739172151,
                                                  0.029597687946635268],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=42)

Since the SVM classifier has been optimized by RandomizedSearchCV, we need to extract the estimator. 

In [10]:
mush_worstModel = mush_worstModel.best_estimator_;
mush_worstModel

SVC(C=3.4370861113902182, degree=4, gamma=0.06200241739172151, kernel='linear')

## Loading the Datasets from files

In [11]:
def load_datasets(path: str):
    # Choco train set
    choco_X_train = unpack(os.path.join(path, constants.filepaths["choc_train-set_samples"]));
    choco_y_train = unpack(os.path.join(path, constants.filepaths["choc_train-set_labels"]));

    # mushroom train set
    mush_X_train = unpack(os.path.join(path, constants.filepaths["mushrooms_train-set_samples"]));
    mush_y_train = unpack(os.path.join(path, constants.filepaths["mushrooms_train-set_labels"]));

    # Choco test set
    choco_X_test = unpack(os.path.join(path, constants.filepaths["choc_test-set_samples"]));
    choco_y_test = unpack(os.path.join(path, constants.filepaths["choc_test-set_labels"]));
    # mushroom test set
    mush_X_test = unpack(os.path.join(path, constants.filepaths["mushrooms_test-set_samples"]));
    mush_y_test = unpack(os.path.join(path, constants.filepaths["mushrooms_test-set_labels"]));

    return choco_X_train, choco_y_train, mush_X_train, mush_y_train, choco_X_test, choco_y_test, mush_X_test, mush_y_test;

try:
    choco_X_train, choco_y_train, mush_X_train, mush_y_train, choco_X_test, choco_y_test, mush_X_test, mush_y_test = load_datasets(constants.ASM1_DIR);
except FileNotFoundError:
    # Using ALT paths for remote / github action
    choco_X_train, choco_y_train, mush_X_train, mush_y_train, choco_X_test, choco_y_test, mush_X_test, mush_y_test = load_datasets(constants.ASM1_DIR_ALT);

### Inspecting the Imported Data

In [12]:
print("-----Chocolate Dataset Training Set-----");
print(f"Size of the samples array in the training set from Chocolate dataset: {len(choco_X_train)}");
print(f"Size of the labels array in the training set from Chocolate dataset: {len(choco_y_train)}");
print(f"Number of features in a sample in the training set from Chocolate dataset: {len(choco_X_train[random.randint(0, len(choco_X_train)-1)])}");
unique, counts = np.unique(choco_y_train, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

print();

print("-----Mushroom Dataset Training Set-----");
print(f"Size of the samples array in the test set from Mushroom dataset: {len(mush_X_train)}");
print(f"Size of the labels array in the test set from Mushroom dataset: {len(mush_y_train)}");
print(f"Number of features in a sample in the test set from Mushroom dataset: {len(mush_X_train[random.randint(0, len(mush_X_train)-1)])}");
unique, counts = np.unique(mush_y_train, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

print();

print("-----Chocolate Dataset Test Set-----");
print(f"Size of the samples array in the test set from Chocolate dataset: {len(choco_X_test)}");
print(f"Size of the labels array in the test set from Chocolate dataset: {len(choco_y_test)}");
print(f"Number of features in a sample in the test set from Chocolate dataset: {len(choco_X_test[random.randint(0, len(choco_X_test)-1)])}");
unique, counts = np.unique(choco_y_test, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

print();

print("-----Mushroom Dataset Test Set-----");
print(f"Size of the samples array in the test set from Mushroom dataset: {len(mush_X_test)}");
print(f"Size of the labels array in the test set from Mushroom dataset: {len(mush_y_test)}");
print(f"Number of features in a sample in the test set from Mushroom dataset: {len(mush_X_test[random.randint(0, len(mush_X_test)-1)])}");
unique, counts = np.unique(mush_y_test, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

-----Chocolate Dataset Training Set-----
Size of the samples array in the training set from Chocolate dataset: 1256
Size of the labels array in the training set from Chocolate dataset: 1256
Number of features in a sample in the training set from Chocolate dataset: 13
Label 'non-user' has: 27 samples.
Label 'user' has: 1229 samples.

-----Mushroom Dataset Training Set-----
Size of the samples array in the test set from Mushroom dataset: 1256
Size of the labels array in the test set from Mushroom dataset: 1256
Number of features in a sample in the test set from Mushroom dataset: 13
Label 'non-user' has: 805 samples.
Label 'user' has: 451 samples.

-----Chocolate Dataset Test Set-----
Size of the samples array in the test set from Chocolate dataset: 629
Size of the labels array in the test set from Chocolate dataset: 629
Number of features in a sample in the test set from Chocolate dataset: 13
Label 'non-user' has: 8 samples.
Label 'user' has: 621 samples.

-----Mushroom Dataset Test Set-

## Apply SHAP Method

A SHAP value is used to represent the impact of each feature on the models’ predictions.

### Instantiating Explainers

In [13]:
treeexmplainer_choco = explainer(
    model=choco_bestModel, # Decision Tree
    data=choco_X_train,
    modelType="tree"
)
treeexmplainer_choco

Start instantiating an explainer.
A tree explainer is instantiated successfully.


In [14]:
linearExplainer_choco = explainer(
    model=choco_worstModel, # SVM
    data=choco_X_train,
    modelType="svm"
);
linearExplainer_choco

Start instantiating an explainer.
A Linear explainer is instantiated successfully.


In [15]:
permExplainer_mush_best = explainer(
    model=mush_bestModel, # MLP
    data=mush_X_train,
    modelType="neural"
)
permExplainer_mush_best

Start instantiating an explainer.
A Permutation Kernel explainer is instantiated successfully.


In [16]:
linearExplainer_mush_worst = explainer(
    model=mush_worstModel, # SVM
    data=mush_X_train,
    modelType="svm"
);
linearExplainer_mush_worst

Start instantiating an explainer.
A Linear explainer is instantiated successfully.


### Calculating SHAP values

In [17]:
# Decision Tree
SHAP_choco_best = treeexmplainer_choco.explain(
    X_test=choco_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_choco_best.shape}");
print(f"The SHAP value for a sample: \n{SHAP_choco_best[random.randint(0, len(SHAP_choco_best)-1)]}");

A tree explainer is found.
Shape of the SHAP values set results: (629, 13, 2)
The SHAP value for a sample: 
[[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.00145422 -0.00145422]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]


In [18]:
# SVM
SHAP_choco_worst = linearExplainer_choco.explain(
    X_test=choco_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_choco_worst.shape}");
print(f"The SHAP value for a sample: \n{SHAP_choco_worst[random.randint(0, len(SHAP_choco_worst)-1)]}");

A Linear explainer is found
Shape of the SHAP values set results: (629, 13)
The SHAP value for a sample: 
[-3.95768718e-04 -8.74319259e-06  3.25230085e-06 -5.50523094e-06
  3.09386075e-05  7.30367327e-07 -5.94554770e-05  4.52305327e-05
 -2.09965038e-05 -1.09213934e-04 -7.20748150e-06  5.87184329e-06
  9.99649849e-09]


In [19]:
# MLP
SHAP_mush_best = permExplainer_mush_best.explain(
    X_test=mush_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_mush_best.shape}");
print(f"The SHAP value for a sample: \n{SHAP_mush_best[random.randint(0, len(SHAP_mush_best)-1)]}");

A Permutation Explainer is found


PermutationExplainer explainer: 630it [02:16,  4.40it/s]                         

Shape of the SHAP values set results: (629, 13)
The SHAP value for a sample: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
# SVM
SHAP_mush_worst = linearExplainer_mush_worst.explain(
    X_test=mush_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_mush_worst.shape}");
print(f"The SHAP value for a sample: \n{SHAP_mush_worst[random.randint(0, len(SHAP_mush_worst)-1)]}");

A Linear explainer is found
Shape of the SHAP values set results: (629, 13)
The SHAP value for a sample: 
[ 3.06110285e+01  3.75229786e-01  1.12839732e-01 -7.82587734e-03
 -3.31578908e-01  1.91051821e-03  9.08253248e-03 -4.84237367e-02
  7.47882911e-01  7.29457037e-03  5.32559278e-03  1.02845116e-01
  1.22453331e-01]


### Store the SHAP values after calculation

In [21]:
np.save("SHAP_choco_best.npy", SHAP_choco_best);
np.save("SHAP_choco_worst.npy", SHAP_choco_worst);
np.save("SHAP_mush_best.npy", SHAP_mush_best);
np.save("SHAP_mush_worst.npy", SHAP_mush_worst);

#### Verify if the Data Dump is successful

In [22]:
test = SHAP_choco_best == np.load("SHAP_choco_best.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [23]:
test = SHAP_choco_worst == np.load("SHAP_choco_worst.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [24]:
test = SHAP_mush_best == np.load("SHAP_mush_best.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [25]:
test = SHAP_mush_worst == np.load("SHAP_mush_worst.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


We have made sure that all the data are dumped to .pkl files (and unpacked) accurately. Therefore, we can proceed with summary plots on another notebook called `CSI5155 Assignment 2 Plots - Kelvin Mock 300453668.ipynb`.